<a href="https://colab.research.google.com/github/Rachanachinthala/rachana_INFO5731_spring2021/blob/main/Rachana_In_class_exercise8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **The eighth in-class-exercise (20 points in total, 3/30/2021)**

The data for this exercise is from the dataset you created from assignment three. Please perform answer the following questions based on your data:

## (1) (10 points) Write a python program to extract the sentiment related terms from the corpus. You may use python package such as polyglot or external lexicon resources in the question. Rank the sentiment related terms by frequency.

In [1]:
!pip install polyglot

     |████████████████████████████████| 133kB 5.4MB/s 
  Created wheel for polyglot: filename=polyglot-16.7.4-py2.py3-none-any.whl size=52557 sha256=e44ab00eb107d16d3bc5940478a46658cac1e32fa9a461e0c9c9fdae6484c207
  Stored in directory: /root/.cache/pip/wheels/5e/91/ef/f1369fdc1203b0a9347d4b24f149b83a305f39ab047986d9da
Successfully built polyglot


In [2]:
!pip install pyicu

     |████████████████████████████████| 235kB 5.3MB/s 
  Created wheel for pyicu: filename=PyICU-2.6-cp37-cp37m-linux_x86_64.whl size=1306408 sha256=8aff4a36ad6b98068f28ef7a6681b8308b7714a37864412c565f8df51eab5e87
  Stored in directory: /root/.cache/pip/wheels/31/21/2f/1c91831e8a93537ab21f6b4b935781b681104635fdb0315791
Successfully built pyicu


In [3]:
!pip install pycld2

     |████████████████████████████████| 41.4MB 97kB/s 
  Created wheel for pycld2: filename=pycld2-0.41-cp37-cp37m-linux_x86_64.whl size=9834303 sha256=bff26f81c980b774ca6c496d17106d3f6ed26b7fbfff715e777bee20e2dd8c2f
  Stored in directory: /root/.cache/pip/wheels/c6/8f/e9/08a1a8932a490175bd140206cd86a3dbcfc70498100de11079
Successfully built pycld2


In [4]:
!pip install morfessor

In [5]:
%%bash
polyglot download sentiment2.en

[polyglot_data] Downloading package sentiment2.en to
[polyglot_data]     /root/polyglot_data...


In [8]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [9]:
import pandas as pd
imdb_df = pd.read_csv('/content/Sentiment data.csv')
imdb_df = imdb_df.loc[:, ~imdb_df.columns.str.contains('^Unnamed')]

**Text Preprocessing**

In [10]:
from nltk.corpus import stopwords
from textblob import TextBlob
imdb_df['Review'] = imdb_df['Review'].apply(lambda x: " ".join(x.lower() for x in x.split()))
imdb_df['Review'] = imdb_df['Review'].str.replace('[^\w\s]','')
stop = stopwords.words('english')
imdb_df['Review'] = imdb_df['Review'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
imdb_df['Review'] = imdb_df['Review'].apply(lambda x: str(TextBlob(x).correct()))

**Extracting words using polyglot**

In [12]:
from polyglot.text import Text
senti_words = []
for value in imdb_df['Review']:
  text = Text(value)
  for a in text.words:
    if a.polarity != 0:
      senti_words.append(a)
senti_words

['brilliant',
 'grotesque',
 'haunt',
 'hard',
 'master',
 'tragic',
 'funny',
 'proper',
 'best',
 'sorry',
 'encourage',
 'encourage',
 'better',
 'respect',
 'like',
 'masterpiece',
 'best',
 'best',
 'dead',
 'best',
 'disturb',
 'violent',
 'corrupt',
 'rich',
 'talent',
 'eager',
 'joke',
 'shake',
 'bitter',
 'comfort',
 'perfect',
 'fearless',
 'impose',
 'dark',
 'joke',
 'accomplish',
 'impose',
 'dark',
 'crime',
 'like',
 'fall',
 'joke',
 'happy',
 'plea',
 'ignore',
 'complaint',
 'desert',
 'better',
 'devil',
 'dark',
 'joke',
 'dark',
 'fall',
 'love',
 'bad',
 'best',
 'comfort',
 'like',
 'horrible',
 'great',
 'inspire',
 'honest',
 'violent',
 'disturb',
 'violent',
 'controversy',
 'great',
 'best',
 'hate',
 'polite',
 'mad',
 'enjoy',
 'masterpiece',
 'plea',
 'enjoy',
 'joke',
 'trouble',
 'lead',
 'delay',
 'worth',
 'joke',
 'desert',
 'festive',
 'regard',
 'entertain',
 'slowly',
 'wonder',
 'joke',
 'good',
 'joke',
 'insane',
 'insane',
 'joke',
 'superio

**Ranking words using frequency**

In [13]:
from collections import Counter
freq = Counter(senti_words)
ranking_df = pd.DataFrame(list(freq.most_common()), columns=['Words', 'Frequencies'])
ranking_df.index = list(range(1, len(freq.most_common())+1))
ranking_df

,Words,Frequencies
1,joke,161
2,like,62
3,good,55
4,great,34
5,best,30
...,...,...
375,fallen,1
376,advantage,1
377,creep,1
378,pleasantly,1


## (2) (10 points) Compare the performance of the following tools in sentiment identification: TextBlob (https://textblob.readthedocs.io/en/dev/), VADER (https://github.com/cjhutto/vaderSentiment), TFIDF-based Support Vector Machine (SVM) (Split your data into training and testing data). Take your own annotation as the standard answers. 

Reference code: https://towardsdatascience.com/fine-grained-sentiment-analysis-in-python-part-1-2697bb111ed4

In [14]:
def Encoder(senti_word):
  senti_encoded = 0
  if senti_word == 'Positive':
    senti_encoded = 1
  elif senti_word == 'Negative':
    senti_encoded = 2
  return senti_encoded

In [15]:
sentiment = []
for senti_word in imdb_df['Sentiment']:
  sentiment.append(Encoder(senti_word))

In [18]:
from textblob import TextBlob
textblob_senti = []
for line in imdb_df['Review']:
  polarity = TextBlob(line).sentiment.polarity
  if polarity > 0:
    textblob_senti.append(1)
  elif polarity < 0:
    textblob_senti.append(2)
  elif polarity == 0.0:
    textblob_senti.append(0)
textblob_df = pd.DataFrame(list(zip(imdb_df['Review'], sentiment, textblob_senti)), columns = ['Reviews', 'Actual', 'Predicted'])
textblob_df

,Reviews,Actual,Predicted
0,every move come truly make impact joaquin perf...,1,1
1,move felt along iso truly relate understand mo...,0,0
2,truly masterpiece best hollywood film 2019 one...,1,1
3,joaquin phoenix give tour de for perform fearl...,1,1
4,time move anticus like end fall short way shor...,1,1
...,...,...,...
101,one worst move ever seen ok may hope surround ...,2,0
102,didn really know much move hadn seen trailer a...,2,1
103,film direct produce poorly opportune take adva...,2,2
104,now move admit first heard joaquin phoenix go ...,1,1


In [19]:
from sklearn.metrics import f1_score, accuracy_score
textblob_accuracy = accuracy_score(textblob_df['Actual'], textblob_df['Predicted'])*100
textblob_f1score = f1_score(textblob_df['Actual'], textblob_df['Predicted'], average = 'macro')
print("Text Blob Accuracy is {0} and F1 Score is {1}".format(textblob_accuracy, textblob_f1score))

Text Blob Accuracy is 49.056603773584904 and F1 Score is 0.3556252819818711


In [20]:
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

In [21]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
vader = SentimentIntensityAnalyzer()
vader_senti = []
for line in imdb_df['Review']:
  polarity = vader.polarity_scores(line)['compound']
  if polarity > 0:
    vader_senti.append(1)
  elif polarity < 0:
    vader_senti.append(2)
  elif polarity == 0.0:
    vader_senti.append(0)
vader_df = pd.DataFrame(list(zip(imdb_df['Review'], sentiment, vader_senti)), columns = ['Reviews', 'Actual', 'Predicted'])
vader_df

/usr/local/lib/python3.7/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


,Reviews,Actual,Predicted
0,every move come truly make impact joaquin perf...,1,1
1,move felt along iso truly relate understand mo...,0,1
2,truly masterpiece best hollywood film 2019 one...,1,1
3,joaquin phoenix give tour de for perform fearl...,1,1
4,time move anticus like end fall short way shor...,1,2
...,...,...,...
101,one worst move ever seen ok may hope surround ...,2,2
102,didn really know much move hadn seen trailer a...,2,1
103,film direct produce poorly opportune take adva...,2,2
104,now move admit first heard joaquin phoenix go ...,1,1


In [22]:
from sklearn.metrics import f1_score, accuracy_score
vader_accuracy = accuracy_score(vader_df['Actual'], vader_df['Predicted'])*100
vader_f1score = f1_score(vader_df['Actual'], vader_df['Predicted'], average = 'macro')
print("Vader Accuracy is {0} and F1 Score is {1}".format(vader_accuracy, vader_f1score))

Vader Accuracy is 50.0 and F1 Score is 0.35047043010752693


In [23]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

In [24]:
TFid_vectorizer = TfidfVectorizer(min_df = 5,
                             max_df = 0.8,
                             sublinear_tf = True,
                             use_idf = True)

In [26]:
Train_a, Test_a, Train_b, Test_b = train_test_split(imdb_df['Review'],imdb_df['Sentiment'],test_size=0.33)
Encoder = LabelEncoder()  
Train_b = Encoder.fit_transform(Train_b)
Test_b = Encoder.fit_transform(Test_b)
Train_a_vectors = TFid_vectorizer.fit_transform(Train_a)
Test_a_vectors = TFid_vectorizer.transform(Test_a)

In [27]:
from sklearn import svm
from sklearn.metrics import classification_report
svm_model = svm.SVC(kernel='linear')
svm_model.fit(Train_a_vectors, Train_b)
predicted = svm_model.predict(Test_a_vectors)
result = classification_report(Test_b, predicted, output_dict=True)
result

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'0': {'f1-score': 0.0, 'precision': 0.0, 'recall': 0.0, 'support': 7},
 '1': {'f1-score': 0.25,
  'precision': 0.16666666666666666,
  'recall': 0.5,
  'support': 6},
 '2': {'f1-score': 0.0, 'precision': 0.0, 'recall': 0.0, 'support': 20},
 '3': {'f1-score': 0.0, 'precision': 0.0, 'recall': 0.0, 'support': 1},
 '4': {'f1-score': 0.0, 'precision': 0.0, 'recall': 0.0, 'support': 1},
 'accuracy': 0.08571428571428572,
 'macro avg': {'f1-score': 0.05,
  'precision': 0.03333333333333333,
  'recall': 0.1,
  'support': 35},
 'weighted avg': {'f1-score': 0.04285714285714286,
  'precision': 0.02857142857142857,
  'recall': 0.08571428571428572,
  'support': 35}}

It can be seen that accuracy is high in vader. F1 score is high in TextBlob. We can conclude that Vader is better than TextBlob and SVM.
From all the above, we can see that accuracy is highest in Vader. F1 score is highest in Text Blob and secong hihest in Vader. By considering all these we can say that Vader is better than text blob and SVM.